In [6]:
import json

import torch
from flock_sdk import FlockModel, FlockSDK
from models.CreditFraudNetMLP import CreditFraudNetMLP
from pandas import DataFrame
from torch import nn
from torch.utils.data import DataLoader, TensorDataset


class ExampleTorchModal(FlockModel):
    def __init__(self, features, dataset_path, epochs=1, lr=0.3) -> None:
        self.epoch = epochs
        self.features = features
        self.lr = lr
        self.dataset_path = dataset_path

        if torch.cuda.is_available():
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")
        self.device = torch.device(device)

        # Initialize the dataset
        self.init_dataset(self.dataset_path)

    def init_dataset(self, dataset_path: str) -> None:
        self.dataset_path = dataset_path
        with open(dataset_path, "r") as f:
            dataset = json.load(f)
        dataset_df = DataFrame.from_records(dataset)

        batch_size = 128

        X_df = dataset_df.iloc[:, :-1]
        y_df = dataset_df.iloc[:, -1]

        X_tensor = torch.tensor(X_df.values, dtype=torch.float32)
        y_tensor = torch.tensor(y_df.values, dtype=torch.float32)

        y_tensor = y_tensor.unsqueeze(1)
        dataframe_in_dataset = TensorDataset(X_tensor, y_tensor)

        self.train_data_loader = DataLoader(
            dataframe_in_dataset,
            batch_size=batch_size,
            shuffle=True,
            drop_last=True,
        )
        self.test_data_loader = DataLoader(
            dataframe_in_dataset,
            batch_size=batch_size,
            shuffle=True,
            drop_last=False,
        )

    def train(self, parameters) -> bytes:
        model = CreditFraudNetMLP(self.features, 1)
        if parameters != None:
            model.load_state_dict(torch.load(io.BytesIO(parameters)))
            model.train()
            optimizer = torch.optim.SGD(
                model.parameters(),
                lr=self.lr,
            )
            criterion = nn.BCELoss()
            model.to(self.device)

            for epoch in range(self.epoch):
                train_loss = 0.0
                train_correct = 0
                train_total = 0
                for inputs, targets in self.train_data_loader:
                    optimizer.zero_grad()

                    inputs, targets = inputs.to(self.device), targets.to(self.device)
                    outputs = model(inputs)

                    loss = criterion(outputs, targets)
                    loss.backward()

                    optimizer.step()

                    train_loss += loss.item() * inputs.size(0)
                    predicted = torch.round(outputs).squeeze()
                    train_total += targets.size(0)
                    train_correct += (predicted == targets.squeeze()).sum().item()

            buffer = io.BytesIO()
            torch.save(model.state_dict(), buffer)
            return buffer.getvalue()

    def evaluate(self, parameters) -> float:
        criterion = nn.BCELoss()

        model = CreditFraudNetMLP(self.features, 1)
        if parameters != None:
            model.load_state_dict(torch.load(io.BytesIO(parameters)))
        model.to(self.device)
        model.eval()

        test_loss = 0.0
        test_correct = 0
        test_total = 0

        with torch.no_grad():
            for inputs, targets in self.test_data_loader:
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)

                test_loss += loss.item() * inputs.size(0)
                predicted = torch.round(outputs).squeeze()
                test_total += targets.size(0)
                test_correct += (predicted == targets.squeeze()).sum().item()

        accuracy = test_correct / test_total
        return accuracy

    def aggregate(self, parameters_list: list[bytes]) -> bytes:
        parameters_list = [
            torch.load(io.BytesIO(parameters)) for parameters in parameters_list
        ]
        averaged_parames_template = parameters_list[0]

        for k in averaged_parames_template.keys():
            temp_w = []
            for local_w in parameters_list:
                temp_w.append(local_w[k])
            averaged_parames_template[k] = sum(temp_w) / len(temp_w)

        buffer = io.BytesIO()
        torch.save(averaged_parames_template, buffer)
        aggregated_parameters = buffer.getvalue()
        return aggregated_parameters


if __name__ == "__main__":
    epochs = 1
    lr = 0.000001
    features = 30
    dataset_path = "data/creditCard.json"
    model = ExampleTorchModal(features, dataset_path, epochs=epochs, lr=lr)
    sdk = FlockSDK(model)
    sdk.run()


FileNotFoundError: [Errno 2] No such file or directory: '/dataset.json'

In [2]:
pip install torch

     -------------------------------------- 159.8/159.8 MB 2.6 MB/s eta 0:00:00
     -------------------------------------- 172.0/172.0 kB 5.1 MB/s eta 0:00:00
     -------------------------------------- 228.5/228.5 MB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 6.2 MB/s eta 0:00:00
     -------------------------------------- 286.4/286.4 kB 8.9 MB/s eta 0:00:00
     -------------------------------------- 536.2/536.2 kB 6.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.17.0 requires huggingface-hub>=0.19.4, which is not installed.
datasets 2.17.0 requires fsspec[http]<=2023.10.0,>=2023.1.0, but you have fsspec 2024.3.1 which is incompatible.


In [4]:
pip install flock_sdk


     -------------------------------------- 60.0/60.0 kB 244.9 kB/s eta 0:00:00
  Using cached Flask-2.3.2-py3-none-any.whl (96 kB)
  Attempting uninstall: flask
    Found existing installation: Flask 3.0.2
    Uninstalling Flask-3.0.2:
      Successfully uninstalled Flask-3.0.2
